# Projet : Enquête ADN — Identifier le coupable
## Contexte
Une scène de crime a été trouvée dans un laboratoire. Un fragment d'ADN a été prélevé et séquencé sur 3 loci (L1, L2, L3).
Vous disposez des profils ADN de 10 suspects (fichier `adn_suspects.csv`) et du profil trouvé sur la scène (`adn_scene.csv`).
Votre objectif : nettoyer les données, comparer les profils, visualiser les résultats et produire une conclusion courte.


---
### Étape 1 — Lecture des données
Lisez les fichiers fournis et affichez les premières lignes.


In [ ]:
# TODO : Charger les fichiers adn_suspects.csv et adn_scene.csv avec pandas
import pandas as pd
suspects = pd.read_csv('Projet_ADN/adn_suspects.csv')
scene = pd.read_csv('Projet_ADN/adn_scene.csv')
display(suspects.head())
display(scene.head())


---
### Étape 2 — Nettoyage des données
Repérez les valeurs manquantes et remplacez-les par la moyenne de chaque colonne (moyenne des loci).


In [ ]:
# TODO : Détecter les NaN et compléter par la moyenne de chaque locus
import numpy as np
print('Valeurs manquantes par colonne :')
print(suspects.isna().sum())
# Remplir les NaN par la moyenne (colonne par colonne)
suspects_clean = suspects.fillna(suspects.mean(numeric_only=True))
display(suspects_clean)


---
### Étape 3a — Analyse comparative
Calculez la différence absolue moyenne entre chaque suspect et l'échantillon trouvé.


In [ ]:
# TODO : Calculer les scores de similarité (différence absolue moyenne)
scene_vals = scene.iloc[0,1:].to_numpy(dtype=float)
scores = {}
for i, row in suspects_clean.iterrows():
    vals = row[['L1','L2','L3']].to_numpy(dtype=float)
    diff = np.abs(vals - scene_vals)
    scores[row['Nom']] = diff.mean()
scores_sorted = dict(sorted(scores.items(), key=lambda x: x[1]))
print('Scores (de la plus petite à la plus grande) :')
print(scores_sorted)


## Étape 3b — Analyse statistique des loci

Avant de comparer les suspects à l’ADN trouvé sur la scène, analysons l'**incertitude des données ADN** observée chez les 10 suspects.

Cette analyse nous permettra de savoir :
- Quels loci sont les plus stables ?
- Quels loci montrent le plus d'incertitude ?

On utilisera des fonctions NumPy pour calculer :
- la **moyenne** (`np.mean()`)
- l’**écart-type** (`np.std()`)

Ces valeurs serviront à créer un graphique à **barres avec barres d’erreur**.


In [ ]:
# TODO : Calculer la moyenne et l'écart-type pour chaque locus
# Indice : utilisez np.mean() et np.std() sur les colonnes du tableau des suspects

# moyennes = ...
# ecarts = ...

# TODO : Afficher ces valeurs
# print("Moyennes :", moyennes)
# print("Écarts-types :", ecarts)


In [ ]:
# TODO : Tracer un graphique avec barres d’erreur
# Indice : plt.bar(x, moyennes, yerr=ecarts, capsize=5, color='lightblue')

# locus = ...
# plt.bar(...)
# plt.xlabel(...)
# plt.ylabel(...)
# plt.title("Incertitude des loci chez les suspects")
# plt.show()


**Interprétation**
- Quel locus semble le plus stable (petit écart-type) ?
- Quel locus montre une grande variabilité (grand écart-type) ?
- Que signifie cette variabilité dans le contexte de l’enquête ?


---
### Étape 4 — Visualisation
Tracez un graphique comparant les loci du suspect le plus probable et de l'échantillon. Ajoutez des barres d'erreur (±2 pb) et une régression linéaire.


In [ ]:
# TODO : Visualiser (barres + erreurs) et tracer une régression linéaire
import matplotlib.pyplot as plt
coupable = list(scores_sorted.keys())[0]
suspect_vals = suspects_clean[suspects_clean['Nom']==coupable][['L1','L2','L3']].to_numpy(dtype=float).flatten()
loci = ['L1','L2','L3']
plt.figure(figsize=(6,4))
plt.bar(loci, suspect_vals, yerr=2, alpha=0.6, label=coupable)
plt.bar(loci, scene_vals, yerr=2, alpha=0.6, label='ADN_scene')
plt.ylabel('Longueur (pb)')
plt.title(f'Comparaison ADN : {coupable} vs scène')
plt.legend()
plt.show()
# Régression
a,b = np.polyfit(scene_vals, suspect_vals, 1)
plt.figure(figsize=(6,4))
plt.plot(scene_vals, suspect_vals, 'o')
xs = np.linspace(min(scene_vals)-5, max(scene_vals)+5, 100)
plt.plot(xs, a*xs+b, '-', label=f'y={a:.2f}x+{b:.2f}')
plt.xlabel('ADN scène (pb)')
plt.ylabel(f'ADN {coupable} (pb)')
plt.legend()
plt.show()


---
### Étape 5 — Résultat final
Enregistrez le nom du coupable probable et les scores dans un fichier texte `resultats_adn.txt`. Rédigez une courte conclusion (3-5 lignes).


In [ ]:
# TODO : Enregistrer les résultats
with open('Projet_ADN/resultats_adn.txt', 'w') as f:
    f.write(f'Coupable probable: {coupable}\n')
    f.write('Scores:\n')
    for name, sc in scores_sorted.items():
        f.write(f'{name}: {sc:.3f}\n')
print('Fichier resultats_adn.txt créé dans le dossier Projet_ADN/')


---
Bon travail ! Remets ce notebook (.ipynb) avec vos graphiques et le fichier `resultats_adn.txt`.
